<h1>CNN for to detect brain tumors using MRI images</h1>

<h3>Этап 1</h3> Импортируем библиотеки

In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

<h3>Этап 2</h3> Определяем директории

In [2]:
DATA_DIR = "data/raw"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
TEST_DIR = "data/test"
categories = ["yes", "no"]

<h3>Этап 3</h3> Создаем директории

In [3]:
for dir_path in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    os.makedirs(dir_path, exist_ok=True)
    for category in categories:
        os.makedirs(os.path.join(dir_path, category), exist_ok=True)

<h3>Этап 4</h3> Распределяем данные по директориям 